# Temporal Analysis of Viral Zoonoses
This Jupyter Notebook builds upon the research conducted by [Tan et al. 2024](https://www.nature.com/articles/s41559-024-02353-4), which, as shown in the image below, uses NCBI Virus data and ancestral sequence reconstruction to identify 216 zoonotic events. Our **aim is to investigate trends in viral zoonoses over time**. Therefore, I incorporated temporal data from NCBI, which was not originally included in the study.

The analysis consists of the following steps:

1. **Zoonotic Events:** Determining the number of zoonotic events per year
   - **Data Preparation**: Load and preprocess data to include only zoonotic events from Supplementary Table 3.
   - **Phylogenetic Analysis**: Identify the closest leaves to host jump nodes using phylogenetic trees provided in the original study.
   - **Metadata Extraction**: Fetch sequence records from NCBI to obtain collection dates for these closest leaves.
   - **Temporal Data Integration**: Use these dates as proxies to estimate the timing of zoonotic events, determining the year of each host jump.

2. **Sequencing Effort:** Determining the sequencing effort for each year
   - **Data Preparation**: Load and preprocess data for the ~60K viral genomes used in the host hump analysis from Supplementary Table 1.
   - **Data Sampling**: Randomly sample 1K sequences from the available dataset to be nice to the NCBI servers.
   - **Year Extraction**: Determine the collection year for each sampled sequence by parsing sequence metadata retrieved from NCBI.

3. **Adjustment:** Adjusting for the sequencing effort and plotting the results
   - **Calculate Effort**: Compute the annual sequencing effort based on the number of sequences recorded each year.
   - **Adjust Events**: Adjust the number of zoonotic events per year by dividing by the sequencing effort.
   - **Visualisation**: Plot the raw number of zoonotic events, the sequencing effort, and the adjusted number of events to visualise trends over time.

<!-- <img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41559-024-02353-4/MediaObjects/41559_2024_2353_Fig3_HTML.png?as=webp" width="800" > -->
<img src="data/example_host_jump.png" width="600" >

## Limitations
While using the closest leaf in the phylogenetic tree with a collection date provides a practical approach to estimate the timing of zoonotic events, the accuracy of this method could be improved by incorporating data from multiple leaves and applying molecular clock methods to estimate divergence times more precisely.

## Conclusion
Although the raw data initially suggest an increase in the number of zoonotic events over time, this trend does not hold after adjusting for sequencing effort. The adjusted data indicate that the apparent increase in zoonotic events is largely due to an increase in sequencing effort rather than a true increase in the number of events.

In [1]:
from Bio import Phylo, Entrez
from operator import itemgetter
from plotly.subplots import make_subplots
from tqdm import tqdm
import logging
import os
import pandas as pd
import plotly.graph_objects as go
import re
import time


# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define constants
Entrez.email = "" # Set your email here

###########################
# Number of zoonotic events
###########################

def get_tree_path(tree_dir, virus):
    return f"{tree_dir}/{virus}.labelled.tree"

def read_tree(tree_path):
    try:
        return Phylo.read(tree_path, "newick")
    except Exception as e:
        logger.error(f"Error reading tree from {tree_path}: {e}")
        return None

def find_closest_leaves(tree, node_name, n=10):
    node = tree.find_any(node_name)
    distances = [(leaf, tree.distance(node, leaf)) for leaf in tree.get_terminals()]
    sorted_leaves = sorted(distances, key=itemgetter(1))[:n]
    return sorted_leaves

def extract_genbank_id(leaf):
    return leaf.name.split("|")[0] if leaf else None

def fetch_record(genbank_id):
    try:
        with Entrez.efetch(db="nucleotide", id=genbank_id, rettype="gb", retmode="text") as handle:
            return handle.read()
    except Exception as e:
        logger.error(f"Error fetching GenBank record for ID {genbank_id}: {e}")
        return None

def parse_collection_date(record):
    if record:
        match = re.search(r'/collection_date="([^"]*)"', record)
        if match:
            return match.group(1)
    return None

def format_dates_and_extract_year(df):
    df["Collection Date"] = pd.to_datetime(df["Collection Date"], errors='coerce')
    df["Year"] = df["Collection Date"].dt.year.astype('Int64', errors='ignore')
    return df

def annotate_dataframe_with_dates(df, tree_dir, n=10):
    """
    Annotate DataFrame with GenBank IDs, distances to closest leaves, and collection dates.
    """
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc='Annotating DataFrame'):
        virus = row["Viral clique"]
        node_name = row["Node name"]
        tree_path = get_tree_path(tree_dir, virus)
        tree = read_tree(tree_path)
        leaf_genbank_id = None
        leaf_distance = None
        collection_date = None

        if tree:
            closest_leaves = find_closest_leaves(tree, node_name, n)
            for leaf, distance in closest_leaves:
                genbank_id = extract_genbank_id(leaf)
                record = fetch_record(genbank_id)
                if record:
                    collection_date = parse_collection_date(record)
                    if collection_date:
                        leaf_genbank_id = genbank_id
                        leaf_distance = distance
                        break  # Stop if we've found a valid GenBank ID with a collection date

        df.loc[i, 'Leaf GenBank ID'] = leaf_genbank_id
        df.loc[i, 'Leaf Distance'] = leaf_distance
        df.loc[i, 'Collection Date'] = collection_date

    return df

#####################################
# Viral cliques and sequencing effort
#####################################

def load_viral_data(xls_path):
    return pd.read_excel(xls_path, sheet_name=0, header=1)

def sample_viral_data(vir_df, sample_size=1000, seed=42):
    return vir_df.sample(n=sample_size, random_state=seed)

def process_genbank_ids(genbank_ids):
    """
    Fetch and process GenBank records for a list of IDs.
    """
    collection_dates = []
    for genbank_id in tqdm(genbank_ids, desc="Fetching GenBank Records"):
        record = fetch_record(genbank_id)
        if record:
            collection_date = parse_collection_date(record)
            collection_dates.append(collection_date)
        else:
            collection_dates.append(None)
        time.sleep(0.1)
    return collection_dates

#####################################
# Combining and plotting the datasets
#####################################

def calculate_sequencing_effort(vir_df):
    return vir_df['Year'].value_counts().sort_index()

def calculate_adjusted_events(zoonotic_df, sequencing_effort):
    zoonotic_events = zoonotic_df['Year'].value_counts().sort_index()
    adjusted_events = zoonotic_events / sequencing_effort
    return adjusted_events.fillna(0)

def create_combined_dataframe(sequencing_effort, zoonotic_events, adjusted_events):
    combined_df = pd.DataFrame({
        'Year': sequencing_effort.index,
        'Sequencing Effort': sequencing_effort.values,
        'Zoonotic Events': zoonotic_events.reindex(sequencing_effort.index, fill_value=0),
        'Adjusted Events': adjusted_events.reindex(sequencing_effort.index, fill_value=0)
    })
    return combined_df

def merge_hover_data(combined_df, zoo_df):
    """
    Merge hover data into the combined DataFrame.
    """
    hover_data = prepare_hover_data(zoo_df)
    return pd.merge(combined_df, hover_data, on='Year', how='left')

def prepare_hover_data(df):
    """
    Aggregates hover data for multiple entries per year into single strings for display.
    """
    hover_data = df.groupby('Year').agg({
        'Viral clique': lambda x: ", ".join(x),
        'Leaf GenBank ID': lambda x: ", ".join(x),
        'Collection Date': lambda x: ", ".join(x.astype(str)),
        'Leaf Distance': lambda x: ", ".join(x.astype(str))
    }).reset_index()
    return hover_data

def plot_combined_data(df):
    """
    Plot the combined data with hover information.
    """
    # Ensure all data is string and handle NaNs
    for col in ['Viral clique', 'Leaf GenBank ID', 'Collection Date', 'Leaf Distance']:
        df[col] = df[col].fillna('N/A').astype(str)
    
    # Create subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces with hover data
    fig.add_trace(
        go.Scatter(
            x=df['Year'],
            y=df['Zoonotic Events'],
            name='Zoonotic Events',
            mode='lines+markers',
            customdata=df[['Viral clique', 'Leaf GenBank ID', 'Collection Date', 'Leaf Distance']],
            hovertemplate="<b>Year:</b> %{x}<br>" +
                          "<b>Number of Events:</b> %{y}<br>" +
                          "<b>Viral Clique:</b> %{customdata[0]}<br>" +
                          "<b>Leaf GenBank ID:</b> %{customdata[1]}<br>" +
                          "<b>Collection Date:</b> %{customdata[2]}<br>" +
                          "<b>Leaf Distance:</b> %{customdata[3]}<br>"
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=df['Year'],
            y=df['Sequencing Effort'],
            name='Sequencing Effort',
            mode='lines+markers'
        ),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(
            x=df['Year'],
            y=df['Adjusted Events'],
            name='Adjusted Zoonotic Events',
            mode='lines+markers'
        ),
        secondary_y=False
    )

    # Add figure title
    fig.update_layout(
        title_text="Zoonotic Events and Sequencing Effort Over Time",
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Year")

    # Set y-axes titles
    fig.update_yaxes(title_text="Number of Zoonotic Events", secondary_y=False)
    fig.update_yaxes(title_text="Sequencing Effort", secondary_y=True)

    # Show the figure
    fig.show()

In [2]:
# Define your data paths
xls_path = "data/41559_2024_2353_MOESM5_ESM.xlsx"
tree_dir = "data/labelled_trees"
vir_csv_path = "data/viral_cliques_with_year.csv"
zoo_csv_path = "data/zoonotic_events_with_year.csv"

# Load the zoonotic events data
if os.path.exists(zoo_csv_path):
    zoo_df = pd.read_csv(zoo_csv_path)
    logger.info("Zoonotic data already annotated.")
else:
    zoo_df = pd.read_excel(xls_path, sheet_name=2, header=4)
    zoo_df = zoo_df[zoo_df["Event type"] == "Zoonotic"]
    df_annotated = annotate_dataframe_with_dates(zoo_df, tree_dir)
    zoo_df = format_dates_and_extract_year(df_annotated)
    zoo_df.to_csv(zoo_csv_path, index=False)
    logger.info("Zoonotic data annotated and saved.")

# Load the viral clique data for sequencing effort
if os.path.exists(vir_csv_path):
    subset_vir_df = pd.read_csv(vir_csv_path)
    logger.info("Viral data already annotated.")
else:
    vir_df = load_viral_data(xls_path)
    subset_vir_df = sample_viral_data(vir_df)
    subset_vir_df['Collection Date'] = process_genbank_ids(subset_vir_df['Accession'].tolist())
    subset_vir_df = format_dates_and_extract_year(subset_vir_df)
    subset_vir_df.to_csv(vir_csv_path, index=False)
    logger.info("Viral data annotated and saved.")
    
# Combine the data
sequencing_effort = calculate_sequencing_effort(subset_vir_df)
adjusted_events = calculate_adjusted_events(zoo_df, sequencing_effort)
combined_df = create_combined_dataframe(sequencing_effort, zoo_df['Year'].value_counts().sort_index(), adjusted_events)
combined_df = merge_hover_data(combined_df, zoo_df)

2024-04-29 15:26:54,929 - __main__ - INFO - Zoonotic data already annotated.
2024-04-29 15:26:54,940 - __main__ - INFO - Viral data already annotated.


In [3]:
plot_combined_data(combined_df)